In [116]:
import numpy as np
import torch
import random
import os
from collections import OrderedDict
from sklearn.preprocessing import OneHotEncoder as Encoder
import numpy as np
import random
import copy
import json
import pandas as pd
!pip install scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
!pip install imbalanced-learn
from sklearn.model_selection import GridSearchCV
from tabulate import tabulate


# Task 1: Calculate the Demographic parity, equal opportunity and predictive rate parity coefficients for this decision rule.

We have two populations Blue (privileged) and Red (unprivileged), with the Blue population being 9 times larger than the Red population.

Individuals from both populations are requesting to attend XAI training to improve competency in this important area. Number of places is limited. The administrators of the training have decided to give priority to enrolling individuals who may need this training in the future, although unfortunately it is difficult to predict who will benefit.

The decision rule adopted:

In the Red group, half of the people will find the skills useful in future and half will not. Administrators randomly allocate 50% of people to training.
in the Blue group, 80% of people will find the training useful in future and 20% will not, although of course it is not known who will find it useful. The administrators have built a predictive model based on user behaviour in predicting for whom it will be useful and whom will not.

In [125]:
table_data_blue= [
    ["Enrolled in training", 60, 5, 65],
    ["Not enrolled in training", 20, 15, 35],
    ["Total", 80, 20, 100]
]
print("Blue")
headers = ["", "Will use XAI", "Will not use XAI", "Total"]
print(tabulate(table_data_blue, headers=headers, tablefmt="fancy_grid"))

Blue
╒══════════════════════════╤════════════════╤════════════════════╤═════════╕
│                          │   Will use XAI │   Will not use XAI │   Total │
╞══════════════════════════╪════════════════╪════════════════════╪═════════╡
│ Enrolled in training     │             60 │                  5 │      65 │
├──────────────────────────┼────────────────┼────────────────────┼─────────┤
│ Not enrolled in training │             20 │                 15 │      35 │
├──────────────────────────┼────────────────┼────────────────────┼─────────┤
│ Total                    │             80 │                 20 │     100 │
╘══════════════════════════╧════════════════╧════════════════════╧═════════╛


In [126]:
table_data_red = [
    ["Enrolled in training", 100/36, 100/36, 100/18],
    ["Not enrolled in training", 100/36, 100/36, 100/18],
    ["Total", 100/18, 100/18, 100/9]
]
print("Red")
headers = ["", "Will use XAI", "Will not use XAI", "Total"]
print(tabulate(table_data_red, headers=headers, tablefmt="fancy_grid"))

Red
╒══════════════════════════╤════════════════╤════════════════════╤══════════╕
│                          │   Will use XAI │   Will not use XAI │    Total │
╞══════════════════════════╪════════════════╪════════════════════╪══════════╡
│ Enrolled in training     │        2.77778 │            2.77778 │  5.55556 │
├──────────────────────────┼────────────────┼────────────────────┼──────────┤
│ Not enrolled in training │        2.77778 │            2.77778 │  5.55556 │
├──────────────────────────┼────────────────┼────────────────────┼──────────┤
│ Total                    │        5.55556 │            5.55556 │ 11.1111  │
╘══════════════════════════╧════════════════╧════════════════════╧══════════╛


Statistical parity coeeficient = (1/2)/(65/100)


Equal opportunity coefficient = (1/2)/(60/80)


Predicitve equality coefficient =(1/2)/(5/20)

In [124]:
Statistical_parity_coeeficient = (1/2)/(65/100)
Equal_opportunity_coefficient = (1/2)/(60/80)
Predicitve_equality_coefficient =(1/2)/(5/20)

print(f'Statistical parity coeeficient: {Statistical_parity_coeeficient}')
print(f'Equal opportunity coefficient: {Equal_opportunity_coefficient}')
print(f'Predicitve equality coefficient: {Predicitve_equality_coefficient}')

Statistical parity coeeficient: 0.7692307692307692
Equal opportunity coefficient: 0.6666666666666666
Predicitve equality coefficient: 2.0


# Task 2: train few models on a selected dataset from https://github.com/ahxt/fair_fairness_benchmark/:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [95]:
credit_risk_path = '/content/drive/My Drive/xai_german_data/raw/german_credit_risk.csv'
credit_risk = pd.read_csv(credit_risk_path,index_col=0)
print(credit_risk.columns)

Index(['Age', 'Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account',
       'Credit amount', 'Duration', 'Purpose', 'Class'],
      dtype='object')


In [96]:
print(credit_risk['Saving accounts'].unique())
print(credit_risk['Checking account'].unique())
print(credit_risk['Housing'].unique())
print(credit_risk['Purpose'].unique())
credit_risk_c=credit_risk.copy()

[nan 'little' 'quite rich' 'rich' 'moderate']
['little' 'moderate' nan 'rich']
['own' 'free' 'rent']
['radio/TV' 'education' 'furniture/equipment' 'car' 'business'
 'domestic appliances' 'repairs' 'vacation/others']


In [97]:
# Mapping the values in 'Saving accounts' column
credit_risk_c['Saving accounts'] = credit_risk_c['Saving accounts'].replace({
    'little': 1,
    'quite rich': 3,
    'rich': 4,
    'moderate': 2,
    pd.NA: 0
})

credit_risk_c['Checking account']=credit_risk_c['Checking account'].replace({
    'little': 1,
    'moderate': 2,
    'rich': 3,
    pd.NA: 0
})

credit_risk_c['Saving accounts']=credit_risk_c['Saving accounts'].replace(np.nan,0)
credit_risk_c['Checking account']=credit_risk_c['Checking account'].replace(np.nan,0)

print(credit_risk_c['Saving accounts'].unique())
print(credit_risk_c['Checking account'].unique())

### One hot encode Sex, Housing, Purpose
credit_risk_c=pd.get_dummies(credit_risk_c,columns=['Sex','Housing','Purpose'])
credit_risk_c2=credit_risk_c.copy()
credit_risk_c3=credit_risk_c.copy()


[0. 1. 3. 4. 2.]
[1. 2. 0. 3.]


<ipython-input-97-65670824c2b9>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  credit_risk_c['Saving accounts'] = credit_risk_c['Saving accounts'].replace({
<ipython-input-97-65670824c2b9>:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  credit_risk_c['Checking account']=credit_risk_c['Checking account'].replace({


In [98]:
# X (features) and Y (target)
X = credit_risk_c[['Age', 'Job', 'Saving accounts', 'Checking account',
                 'Credit amount', 'Duration', 'Sex_female', 'Sex_male',
                 'Housing_free', 'Housing_own', 'Housing_rent',
                 'Purpose_business', 'Purpose_car', 'Purpose_domestic appliances',
                 'Purpose_education', 'Purpose_furniture/equipment',
                 'Purpose_radio/TV', 'Purpose_repairs', 'Purpose_vacation/others']]

Y = credit_risk_c['Class']

In [99]:
# training (80%) and testing (20%) sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")


Training data shape: (800, 19)
Testing data shape: (200, 19)


# 1. Train a model for the selected dataset: Random Forest Classifier

In [101]:
param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [None, 10, 30],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=1, scoring='accuracy')

grid_search.fit(X_train, Y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

best_rf_model = grid_search.best_estimator_
Y_pred_best_rf = best_rf_model.predict(X_test)
print("Test Set Accuracy:", accuracy_score(Y_test, Y_pred_best_rf))
print("\nClassification Report:\n", classification_report(Y_test, Y_pred_best_rf))


Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 500}
Best Cross-Validation Accuracy: 0.75
Test Set Accuracy: 0.745

Classification Report:
               precision    recall  f1-score   support

           1       0.77      0.91      0.83       141
           2       0.62      0.34      0.44        59

    accuracy                           0.74       200
   macro avg       0.70      0.63      0.64       200
weighted avg       0.73      0.74      0.72       200



We choose the best RF classifier

In [102]:
# Random Forest Classifier model 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 500
rf_model=RandomForestClassifier(n_estimators=500, max_depth=10, min_samples_leaf=1,min_samples_split=5, random_state=42)

rf_model.fit(X_train, Y_train)
Y_pred_rf = rf_model.predict(X_test)

print("Random Forest Model Evaluation:")
print("Accuracy:", accuracy_score(Y_test, Y_pred_rf))
print("\nClassification Report:\n", classification_report(Y_test, Y_pred_rf))

acc_first_rf=accuracy_score(Y_test, Y_pred_rf)

Random Forest Model Evaluation:
Accuracy: 0.745

Classification Report:
               precision    recall  f1-score   support

           1       0.77      0.91      0.83       141
           2       0.62      0.34      0.44        59

    accuracy                           0.74       200
   macro avg       0.70      0.63      0.64       200
weighted avg       0.73      0.74      0.72       200



 # 2. For the selected protected attribute (gender) calculate the following fairness coefficients: Statistical parity, Equal opportunity, Predictive parity.

In [103]:
credit_risk_c['Prediction']=rf_model.predict(X)
parity_df=credit_risk_c[['Sex_female','Sex_male','Class','Prediction']]
parity_df.head()

females_count=len(parity_df[parity_df['Sex_female']==1])
males_count=len(parity_df[parity_df['Sex_male']==1])

females_predict2=len(parity_df[(parity_df['Sex_female']==1) & (parity_df['Prediction']==2)])
males_predict2=len(parity_df[(parity_df['Sex_male']==1) & (parity_df['Prediction']==2)])

females_class2=len(parity_df[(parity_df['Sex_female']==1) & (parity_df['Class']==2)])
males_class2=len(parity_df[(parity_df['Sex_male']==1) & (parity_df['Class']==2)])

females_class1=len(parity_df[(parity_df['Sex_female']==1) & (parity_df['Class']==1)])
males_class1=len(parity_df[(parity_df['Sex_male']==1) & (parity_df['Class']==1)])

females_class2predict2=len(parity_df[(parity_df['Sex_female']==1) & (parity_df['Prediction']==2) & (parity_df['Class']==2)])
males_class2predcit2=len(parity_df[(parity_df['Sex_male']==1) & (parity_df['Prediction']==2) & (parity_df['Class']==2)])

females_class1predict2=len(parity_df[(parity_df['Sex_female']==1) & (parity_df['Prediction']==2) & (parity_df['Class']==1)])
males_class1predcit2=len(parity_df[(parity_df['Sex_male']==1) & (parity_df['Prediction']==2) & (parity_df['Class']==1)])


prob_predict2_females=females_predict2/females_count
prob_predict2_males=males_predict2/males_count

prob_eq_female=females_class2predict2/females_class2
prob_eq_male=males_class2predcit2/males_class2

prob_pred_eq_female=females_class1predict2/females_class1
prob_pred_eq_male=males_class1predcit2/males_class1

equal_opportunity_coef_rf=prob_eq_female/prob_eq_male
statistical_parity_coef_rf=prob_predict2_females/prob_predict2_males
predictive_equality_coef_rf=prob_pred_eq_female/prob_pred_eq_male
print(f'Statistical parity coefficient: {statistical_parity_coef_rf}')
print(f'Equal opportunity coefficient: {equal_opportunity_coef_rf}')
print(f'Predictive equality coefficient: {predictive_equality_coef_rf}')


Statistical parity coefficient: 1.5975546305931323
Equal opportunity coefficient: 1.3065346853371962
Predictive equality coefficient: 0.9930348258706467


# 3. Train another model (Gradient Boosting Classifier) and see how the coefficients Statistical parity, Equal opportunity, Predictive parity behave for it. Are they different/similar?

In [104]:
from sklearn.ensemble import GradientBoostingClassifier
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, Y_train)
Y_pred_gb = gb_model.predict(X_test)

print("Gradient Boosting Test Accuracy:", accuracy_score(Y_test, Y_pred_gb))
print("\nClassification Report:\n", classification_report(Y_test, Y_pred_gb))
acc_first_gb=accuracy_score(Y_test, Y_pred_gb)

Gradient Boosting Test Accuracy: 0.76

Classification Report:
               precision    recall  f1-score   support

           1       0.78      0.93      0.85       141
           2       0.68      0.36      0.47        59

    accuracy                           0.76       200
   macro avg       0.73      0.64      0.66       200
weighted avg       0.75      0.76      0.73       200



In [105]:
credit_risk_c2['Prediction']=gb_model.predict(X)
parity_df=credit_risk_c2[['Sex_female','Sex_male','Class','Prediction']]

females_count=len(parity_df[parity_df['Sex_female']==1])
males_count=len(parity_df[parity_df['Sex_male']==1])

females_predict2=len(parity_df[(parity_df['Sex_female']==1) & (parity_df['Prediction']==2)])
males_predict2=len(parity_df[(parity_df['Sex_male']==1) & (parity_df['Prediction']==2)])

females_class2=len(parity_df[(parity_df['Sex_female']==1) & (parity_df['Class']==2)])
males_class2=len(parity_df[(parity_df['Sex_male']==1) & (parity_df['Class']==2)])

females_class1=len(parity_df[(parity_df['Sex_female']==1) & (parity_df['Class']==1)])
males_class1=len(parity_df[(parity_df['Sex_male']==1) & (parity_df['Class']==1)])

females_class2predict2=len(parity_df[(parity_df['Sex_female']==1) & (parity_df['Prediction']==2) & (parity_df['Class']==2)])
males_class2predcit2=len(parity_df[(parity_df['Sex_male']==1) & (parity_df['Prediction']==2) & (parity_df['Class']==2)])

females_class1predict2=len(parity_df[(parity_df['Sex_female']==1) & (parity_df['Prediction']==2) & (parity_df['Class']==1)])
males_class1predcit2=len(parity_df[(parity_df['Sex_male']==1) & (parity_df['Prediction']==2) & (parity_df['Class']==1)])


prob_predict2_females=females_predict2/females_count
prob_predict2_males=males_predict2/males_count

prob_eq_female=females_class2predict2/females_class2
prob_eq_male=males_class2predcit2/males_class2

prob_pred_eq_female=females_class1predict2/females_class1
prob_pred_eq_male=males_class1predcit2/males_class1

equal_opportunity_coef_gb=prob_eq_female/prob_eq_male
statistical_parity_coef_gb=prob_predict2_females/prob_predict2_males
predictive_equality_coef_gb=prob_pred_eq_female/prob_pred_eq_male
print(f'Statistical parity coefficient: {statistical_parity_coef_gb}')
print(f'Equal opportunity coefficient: {equal_opportunity_coef_gb}')
print(f'Predictive equality coefficient: {predictive_equality_coef_gb}')

Statistical parity coefficient: 1.5028949545078578
Equal opportunity coefficient: 1.1853750674581758
Predictive equality coefficient: 1.6550580431177448


# 4. Apply the selected bias mitigation technique (like data balancing) on the first model. Check how Statistical parity, Equal opportunity, Predictive parity coefficients behave after this mittigation.

In [106]:
# Data balancing
print("Females",females_count,"Males",males_count)
print("Females class 2",females_class2,"Males class 2",males_class2)
print("Females class 1",females_class1,"Males class 1",males_class1)

Females 310 Males 690
Females class 2 109 Males class 2 191
Females class 1 201 Males class 1 499


In [107]:
male_proportion = males_class2 / males_class1
print(f"Proportion of males in class 2 to males in class 1: {male_proportion}")

target_females_class2 = int(females_class1 * male_proportion)
print(f"Target number of females in class 2: {target_females_class2}")

class2_females = credit_risk_c[(credit_risk_c['Sex_female'] == 1) & (credit_risk_c['Class'] == 2)]
class2_males = credit_risk_c[(credit_risk_c['Sex_male'] == 1) & (credit_risk_c['Class'] == 2)]
class1_females = credit_risk_c[(credit_risk_c['Sex_female'] == 1) & (credit_risk_c['Class'] == 1)]
class1_males = credit_risk_c[(credit_risk_c['Sex_male'] == 1) & (credit_risk_c['Class'] == 1)]


sampled_females_class2 = class2_females.sample(n=76, random_state=42)
print(f"Sampled females in class 2: {sampled_females_class2.shape[0]}")

combined_df = pd.concat([sampled_females_class2, class2_males, class1_males, class1_females], axis=0)

females_class2r=len(combined_df[(combined_df['Sex_female']==1) & (combined_df['Class']==2)])
males_class2r=len(combined_df[(combined_df['Sex_male']==1) & (combined_df['Class']==2)])
females_class1r=len(combined_df[(combined_df['Sex_female']==1) & (combined_df['Class']==1)])
males_class1r=len(combined_df[(combined_df['Sex_male']==1) & (combined_df['Class']==1)])

print(females_class2r)
print(males_class2r)
print(females_class1r)
print(males_class1r)

Proportion of males in class 2 to males in class 1: 0.38276553106212424
Target number of females in class 2: 76
Sampled females in class 2: 76
76
191
201
499


In [108]:
X = combined_df[['Age', 'Job', 'Saving accounts', 'Checking account',
                 'Credit amount', 'Duration', 'Sex_female', 'Sex_male',
                 'Housing_free', 'Housing_own', 'Housing_rent',
                 'Purpose_business', 'Purpose_car', 'Purpose_domestic appliances',
                 'Purpose_education', 'Purpose_furniture/equipment',
                 'Purpose_radio/TV', 'Purpose_repairs', 'Purpose_vacation/others']]

Y = combined_df['Class']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

# Create the Random Forest Classifier model 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 500
rf_model=RandomForestClassifier(n_estimators=500, max_depth=10, min_samples_leaf=1,min_samples_split=5, random_state=42)
rf_model.fit(X_train, Y_train)
Y_pred_rf = rf_model.predict(X_test)

print("Random Forest Model Evaluation:")
print("Accuracy:", accuracy_score(Y_test, Y_pred_rf))
print("\nClassification Report:\n", classification_report(Y_test, Y_pred_rf))

acc_first_improved=accuracy_score(Y_test, Y_pred_rf)

Training data shape: (773, 19)
Testing data shape: (194, 19)
Random Forest Model Evaluation:
Accuracy: 0.7783505154639175

Classification Report:
               precision    recall  f1-score   support

           1       0.81      0.93      0.86       147
           2       0.58      0.30      0.39        47

    accuracy                           0.78       194
   macro avg       0.69      0.61      0.63       194
weighted avg       0.75      0.78      0.75       194



In [110]:
X = credit_risk_c3[['Age', 'Job', 'Saving accounts', 'Checking account',
                 'Credit amount', 'Duration', 'Sex_female', 'Sex_male',
                 'Housing_free', 'Housing_own', 'Housing_rent',
                 'Purpose_business', 'Purpose_car', 'Purpose_domestic appliances',
                 'Purpose_education', 'Purpose_furniture/equipment',
                 'Purpose_radio/TV', 'Purpose_repairs', 'Purpose_vacation/others']]

Y = credit_risk_c3['Class']

credit_risk_c3['Prediction']=rf_model.predict(X)
parity_df=credit_risk_c3[['Sex_female','Sex_male','Class','Prediction']]

females_count=len(parity_df[parity_df['Sex_female']==1])
males_count=len(parity_df[parity_df['Sex_male']==1])

females_predict2=len(parity_df[(parity_df['Sex_female']==1) & (parity_df['Prediction']==2)])
males_predict2=len(parity_df[(parity_df['Sex_male']==1) & (parity_df['Prediction']==2)])

females_class2=len(parity_df[(parity_df['Sex_female']==1) & (parity_df['Class']==2)])
males_class2=len(parity_df[(parity_df['Sex_male']==1) & (parity_df['Class']==2)])

females_class1=len(parity_df[(parity_df['Sex_female']==1) & (parity_df['Class']==1)])
males_class1=len(parity_df[(parity_df['Sex_male']==1) & (parity_df['Class']==1)])

females_class2predict2=len(parity_df[(parity_df['Sex_female']==1) & (parity_df['Prediction']==2) & (parity_df['Class']==2)])
males_class2predcit2=len(parity_df[(parity_df['Sex_male']==1) & (parity_df['Prediction']==2) & (parity_df['Class']==2)])

females_class1predict2=len(parity_df[(parity_df['Sex_female']==1) & (parity_df['Prediction']==2) & (parity_df['Class']==1)])
males_class1predcit2=len(parity_df[(parity_df['Sex_male']==1) & (parity_df['Prediction']==2) & (parity_df['Class']==1)])


prob_predict2_females=females_predict2/females_count
prob_predict2_males=males_predict2/males_count

prob_eq_female=females_class2predict2/females_class2
prob_eq_male=males_class2predcit2/males_class2

prob_pred_eq_female=females_class1predict2/females_class1
prob_pred_eq_male=males_class1predcit2/males_class1

equal_opportunity_coef_improved=prob_eq_female/prob_eq_male
statistical_parity_coef_improved=prob_predict2_females/prob_predict2_males
predictive_equality_coef_improved=prob_pred_eq_female/prob_pred_eq_male
print(f'Statistical parity coefficient: {statistical_parity_coef_improved}')
print(f'Equal opportunity coefficient: {equal_opportunity_coef_improved}')
print(f'Predictive equality coefficient: {predictive_equality_coef_improved}')

Statistical parity coefficient: 1.1824596774193548
Equal opportunity coefficient: 0.9345565749235475
Predictive equality coefficient: 1.2412935323383085


# 5. Compare the quality (performance) of the three models with their fairness coefficients. Is there any correlation/trade off?

In [117]:
data = {
    'Model': ['Random Forest', 'Gradient Boosting', 'Improved Random Forest Model'],
    'Accuracy on test set': [acc_first_rf, acc_first_gb, acc_first_improved],
    'Statistical Parity Coefficient': [
        statistical_parity_coef_rf,
        statistical_parity_coef_gb,
        statistical_parity_coef_improved
    ],
    'Equal Opportunity Coefficient': [
        equal_opportunity_coef_rf,
        equal_opportunity_coef_gb,
        equal_opportunity_coef_improved
    ],
    'Predictive Equality Coefficient': [
        predictive_equality_coef_rf,
        predictive_equality_coef_gb,
        predictive_equality_coef_improved
    ]
}

metrics_df = pd.DataFrame(data)
table = tabulate(metrics_df, headers='keys', tablefmt='fancy_grid')
print(table)

╒════╤══════════════════════════════╤════════════════════════╤══════════════════════════════════╤═════════════════════════════════╤═══════════════════════════════════╕
│    │ Model                        │   Accuracy on test set │   Statistical Parity Coefficient │   Equal Opportunity Coefficient │   Predictive Equality Coefficient │
╞════╪══════════════════════════════╪════════════════════════╪══════════════════════════════════╪═════════════════════════════════╪═══════════════════════════════════╡
│  0 │ Random Forest                │               0.745    │                          1.59755 │                        1.30653  │                          0.993035 │
├────┼──────────────────────────────┼────────────────────────┼──────────────────────────────────┼─────────────────────────────────┼───────────────────────────────────┤
│  1 │ Gradient Boosting            │               0.76     │                          1.50289 │                        1.18538  │                          1.6

# 6. Comment on the results

I trained three models: a Random Forest Classifier, a Gradient Boosting Classifier, and an improved model with bias mitigation (using sex as the protected attribute) by applying data balancing. I ensured that the proportion of:

$ \frac{\text{Females in class 2}}{\text{Females in class 1}} = \frac{\text{Males in class 2}}{\text{Males in class 1}} $

remained equal, and then trained a Random Forest Classifier on the balanced data.

The improved Random Forest model achieved a higher accuracy score, although it was tested on a different test dataset. In terms of fairness metrics, it performed better in terms of the Statistical Parity Coefficient and Equal Opportunity Coefficient compared to the other models. However, it had a worse Predictive Equality Coefficient than the original Random Forest Classifier. This demonstrates that it's challenging to improve all three fairness metrics simultaneously.



